In [19]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [20]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\12974\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [31]:
import pandas as pd

# Загрузка данных
df = pd.read_csv(r"C:\Users\12974\Desktop\sms+spam+collection\SMSSpamCollection", sep='\t', header=None, names=["label", "text"])

# Отображение первых нескольких строк набора данных
print(df.head())


  label                                               text
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [32]:
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Инициализация стеммера
stemmer = SnowballStemmer("russian")

# Загрузка списка стоп-слов
stop_words = set(stopwords.words("russian"))

# Функция предварительной обработки текста
def preprocess_text(text):
    # Очистка текста от ненужных символов и разбиение на слова
    words = re.findall(r'\b\w+\b', text.lower())
    # Удаление стоп-слов и применение стемминга
    words = [stemmer.stem(word) for word in words if word not in stop_words]
    # Сборка токенов обратно в текст
    return " ".join(words)

# Применение функции предварительной обработки к столбцу с текстом
df["text"] = df["text"].apply(preprocess_text)

# Вывод нескольких строк обновленного набора данных
for index, row in df.iterrows():
    print("label:", row["label"])
    print("text:", row["text"])
    print()


label: ham
text: go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat

label: ham
text: ok lar joking wif u oni

label: spam
text: free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive entry question std txt rate t c s apply 08452810075over18 s

label: ham
text: u dun say so early hor u c already then say

label: ham
text: nah i don t think he goes to usf he lives around here though

label: spam
text: freemsg hey there darling it s been 3 week s now and no word back i d like some fun you up for it still tb ok xxx std chgs to send 1 50 to rcv

label: ham
text: even my brother is not like to speak with me they treat me like aids patent

label: ham
text: as per your request melle melle oru minnaminunginte nurungu vettam has been set as your callertune for all callers press 9 to copy your friends callertune

label: spam
text: winner as a valued network customer you have been selected to receivea 900 

In [33]:
from sklearn.model_selection import train_test_split

# Разделение данных на признаки (X) и целевую переменную (y)
X = df['text']
y = df['label']

# Разделение данных на тренировочный и тестовый наборы (80% - тренировочный, 20% - тестовый)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Проверка размеров наборов данных
print("Размер тренировочного набора данных:", X_train.shape[0])
print("Размер тестового набора данных:", X_test.shape[0])


Размер тренировочного набора данных: 4457
Размер тестового набора данных: 1115


In [34]:
from sklearn.feature_extraction.text import CountVectorizer

# Создание экземпляра CountVectorizer
count_vectorizer = CountVectorizer()

# Преобразование тренировочного и тестового наборов данных с помощью CountVectorizer
X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

# Вывод размеров преобразованных наборов данных
print("Размер матрицы признаков тренировочного набора (CountVectorizer):", X_train_count.shape)
print("Размер матрицы признаков тестового набора (CountVectorizer):", X_test_count.shape)


Размер матрицы признаков тренировочного набора (CountVectorizer): (4457, 7702)
Размер матрицы признаков тестового набора (CountVectorizer): (1115, 7702)


In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Создание экземпляра TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Преобразование тренировочного и тестового наборов данных с помощью TfidfVectorizer
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Вывод размеров преобразованных наборов данных
print("Размер матрицы признаков тренировочного набора (TfidfVectorizer):", X_train_tfidf.shape)
print("Размер матрицы признаков тестового набора (TfidfVectorizer):", X_test_tfidf.shape)


Размер матрицы признаков тренировочного набора (TfidfVectorizer): (4457, 7702)
Размер матрицы признаков тестового набора (TfidfVectorizer): (1115, 7702)


In [36]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Инициализация и обучение классификатора SVC на матрице признаков от CountVectorizer
svc_count = SVC()
svc_count.fit(X_train_count, y_train)

# Предсказание меток для тестового набора данных
y_pred_svc_count = svc_count.predict(X_test_count)

# Оценка точности классификации
accuracy_svc_count = accuracy_score(y_test, y_pred_svc_count)
print("Точность классификации с использованием CountVectorizer и SVC:", accuracy_svc_count)


Точность классификации с использованием CountVectorizer и SVC: 0.9865470852017937


In [37]:
from sklearn.linear_model import LogisticRegression

# Инициализация и обучение классификатора LogisticRegression на матрице признаков от CountVectorizer
lr_count = LogisticRegression()
lr_count.fit(X_train_count, y_train)

# Предсказание меток для тестового набора данных
y_pred_lr_count = lr_count.predict(X_test_count)

# Оценка точности классификации
accuracy_lr_count = accuracy_score(y_test, y_pred_lr_count)
print("Точность классификации с использованием CountVectorizer и LogisticRegression:", accuracy_lr_count)


Точность классификации с использованием CountVectorizer и LogisticRegression: 0.9883408071748879


In [40]:
# Инициализация и обучение классификатора SVC на матрице признаков от TfidfVectorizer
svc_tfidf = SVC()
svc_tfidf.fit(X_train_tfidf, y_train)

# Предсказание меток для тестового набора данных
y_pred_svc_tfidf = svc_tfidf.predict(X_test_tfidf)

# Оценка точности классификации
accuracy_svc_tfidf = accuracy_score(y_test, y_pred_svc_tfidf)
print("Точность классификации с использованием TfidfVectorizer и SVC:", accuracy_svc_tfidf)

# Инициализация и обучение классификатора LogisticRegression на матрице признаков от TfidfVectorizer
lr_tfidf = LogisticRegression()
lr_tfidf.fit(X_train_tfidf, y_train)

# Предсказание меток для тестового набора данных
y_pred_lr_tfidf = lr_tfidf.predict(X_test_tfidf)

# Оценка точности классификации
accuracy_lr_tfidf = accuracy_score(y_test, y_pred_lr_tfidf)
print("Точность классификации с использованием TfidfVectorizer и LogisticRegression:", accuracy_lr_tfidf)



Точность классификации с использованием TfidfVectorizer и SVC: 0.989237668161435
Точность классификации с использованием TfidfVectorizer и LogisticRegression: 0.9766816143497757
